# Terraces in the first Movement of Sibelius Symphony No. 6

Here goes some nice abstract of the project.

## Introduction

Here goes some nice intro. More text...


In [7]:
print("Hello from python!")


Hello from python!


## Evaluation

* loading data from oma.digital
* login via tokens 



In [69]:
__author__ = "Martin Aigner"

import requests
import json


class Oma:
    def __init__(self, endpoint, access_token, verify_certificate=True):
        self.endpoint = endpoint
        self.access_token = access_token
        self.verify_certificate = verify_certificate
        self.headers = {
            'User-Agent': 'oma.py',
            'Authorization': "Bearer " + access_token
        }
        self.config = {
            "api_path": "api/",
            "cache_dir": "omacache"
        }
        
    def recording(self, id):
        r = Recording(id, self)
        r.get()
        return r
    
    def composer(self, id):
        c = Composer(id, self)
        c.get()
        return c

    def annotation_session(self, id):
        s = AnnotationSession(id, self)
        s.get()
        return s



class ApiObject:
    def __init__(self):
        print("ApiObject constructor")
        self.__data_cache = None
        self.oma_login = None
        self.params = {}

    def get(self):
        print(self.params)
        self.params[self.objectName] = self.id
        url = self.oma_login.endpoint + self.oma_login.config["api_path"] + self.objectName
        response = requests.post(url, headers=self.oma_login.headers, data=self.params, verify=self.oma_login.verify_certificate)
        # print(response.content.decode())
        content = json.loads(response.content.decode())
        self.__data_cache = content

    def dictionary(self):
        if self.__data_cache is None:
            self.get()
        return self.__data_cache


class AnnotationSession(ApiObject):
    def __init__(self, id, oma_login):
            self.id = id
            self.objectName = "session"
            self.oma_login = oma_login

        
class Recording(ApiObject):
    def __init__(self, id, oma_login):
        self.id = id
        self.objectName = "recording"
        self.oma_login = oma_login
        self.params = {"method": "get"}

    def get_annotation_sessions(self):
        session = ApiObject()
        session.params["method"] = "findBy"
        session.params["findBy"] = "recording"
        session.params["recording"] = self.id
        session.objectName = "annotation"
        session.id = 0
        session.oma_login = self.oma_login
        session.get()
        return session

        
class Composer(ApiObject):
    def __init__(self, id, oma_login):
        self.id = id
        self.objectName = "composer"
        self.oma_login = oma_login
        self.params = {"method": "get"}

        

url = "http://localhost:8080/"
token = "4fdnupfvk379b52n00m35n1s5jqtc31s"

oma = Oma(url, token)
recording = oma.recording(187642)
#oma.composer(187593)

sessions = recording.get_annotation_sessions()

sessions_data = sessions.dictionary()




#session = oma.annotation_session(187979)

{'method': 'get'}
ApiObject constructor
{'recording': 187642, 'findBy': 'recording', 'method': 'findBy'}


[{'annotationId': 187980,
  'annotationSession': 187979,
  'barNumber': 1,
  'beatNumber': 1,
  'intValue': None,
  'momentOfPerception': 2.600634921,
  'recordingId': 187642,
  'sessionName': 'Upload of Sibelius_Sym6_Karajan_1955_ganzt_fertig_I.csv',
  'stringValue': None,
  'subDivision': None,
  'type': 'Tap'},
 {'annotationId': 187981,
  'annotationSession': 187979,
  'barNumber': 2,
  'beatNumber': 1,
  'intValue': None,
  'momentOfPerception': 5.026235828,
  'recordingId': 187642,
  'sessionName': 'Upload of Sibelius_Sym6_Karajan_1955_ganzt_fertig_I.csv',
  'stringValue': None,
  'subDivision': None,
  'type': 'Tap'},
 {'annotationId': 187982,
  'annotationSession': 187979,
  'barNumber': 3,
  'beatNumber': 1,
  'intValue': None,
  'momentOfPerception': 7.689795918,
  'recordingId': 187642,
  'sessionName': 'Upload of Sibelius_Sym6_Karajan_1955_ganzt_fertig_I.csv',
  'stringValue': None,
  'subDivision': None,
  'type': 'Tap'},
 {'annotationId': 187983,
  'annotationSession': 187